In [1]:
import os
import sys
sys.path.insert(0, os.path.dirname(os.getcwd()))

In [2]:
import pandas as pd
from utils.constants import STOCK_SET, MARKET_KEYWORD_SET, Fl_PARAM
from utils.nyt_api_helpers import (
    get_project_root, gen_mkt_filter, 
    gen_stock_filter,make_api_call, get_project_root
)
key = os.getenv("NYT_API_KEY")


In [2]:
data_dir = get_project_root().joinpath("data/raw_articles_decprecated.csv")
dat = pd.read_csv(data_dir, index_col=0)
dat.head()


,abstract,web_url,snippet,lead_paragraph,source,headline,pub_date,meta.article_type,meta.arguments
0,As companies reported their latest quarterly e...,https://www.nytimes.com/2023/08/10/business/ri...,NaN,Companies that sell food and other household s...,The New York Times,How Long Will Companies Keep Raising Prices on...,2023-08-10 09:01:06+00:00,general_mkt,"('healthcare sector', 'energy sector', 'financ..."
1,Proposed new limits come as corporations may s...,https://www.nytimes.com/2022/03/28/business/de...,NaN,When the Biden administration unveils its annu...,The New York Times,Biden Renews Pushback Against Stock Buybacks,2022-03-28 11:36:41+00:00,general_mkt,"('retail sales', 'global trade', 'supply chain..."
2,The supposed link between buybacks and inequal...,https://www.nytimes.com/2021/09/18/business/de...,NaN,The DealBook newsletter delves into a single t...,The New York Times,Why Taxing Stock Buybacks Is the Wrong Fix for...,2021-09-18 12:00:04+00:00,general_mkt,"('retail sales', 'global trade', 'supply chain..."
3,"On his first day back in the top job, Howard S...",https://www.nytimes.com/2022/04/04/business/st...,NaN,As Starbucks faces a rising wave of employee u...,The New York Times,Starbucks’s new C.E.O. scraps stock buybacks t...,2022-04-04 07:01:03+00:00,general_mkt,"('retail sales', 'global trade', 'supply chain..."
4,"The job cuts, which amount to less than 5 perc...",https://www.nytimes.com/2023/01/18/business/mi...,NaN,Microsoft on Wednesday became the latest addit...,The New York Times,"Microsoft to Lay Off 10,000 Workers as It Look...",2023-01-18 14:15:58+00:00,stock,"('Microsoft', 'MSFT')"


In [3]:
dat['meta.arguments'].value_counts()

meta.arguments
('Facebook', 'META')                                                                                 2010
('Amazon', 'AMZN')                                                                                    470
('Google', 'GOOGL')                                                                                   422
('Tesla', 'TSLA')                                                                                     244
('Microsoft', 'MSFT')                                                                                 138
('NVIDIA', 'NVDA')                                                                                     22
('retail sales', 'global trade', 'supply chain', 'corporate earnings', 'stock buybacks')                3
('healthcare sector', 'energy sector', 'financial sector', 'industrial sector', 'consumer goods')       1
Name: count, dtype: int64

In [16]:
hits_list = []
for mkt in MARKET_KEYWORD_SET:
    filt = gen_mkt_filter(*mkt)
    payload = {
        'begin_date': "2021-01-01",
        'fl': Fl_PARAM
    }
    res = make_api_call(payload,key,filt, time_delay=True)
    hits_list.append({
        "mkt_keywords": mkt,
        "hits": res['num_hits'],
        'status': res['meta']['status_code']
    })

mkt_hit_dat = pd.json_normalize(hits_list)
mkt_hit_dat.to_csv(get_project_root().joinpath("data/mkt_hits.csv"))
    

In [19]:
mkt_hit_dat

,mkt_keywods,hits,status
0,"(earnings, revenue, profits, growth, sales, ma...",0,200
1,"(economic outlook, GDP, inflation, interest ra...",0,200
2,"(fiscal policy, unemployment, labor market, co...",0,200
3,"(healthcare sector, energy sector, financial s...",1,200
4,"(retail sales, global trade, supply chain, cor...",3,200
5,"(dividends, market volatility, investor sentim...",0,200
6,"(regulatory changes, geopolitical, climate cha...",16,200
7,"(corporate governance, cybersecurity, innovati...",0,200


In [9]:
hits_list = []
for stock, ticker in STOCK_SET:
    filt = gen_stock_filter(stock_name=stock,ticker=ticker)
    payload = {
        'begin_date': "2021-01-01",
        'fl': Fl_PARAM
    }
    res = make_api_call(payload,key, filt, time_delay=True)
    hits_list.append({
        "stock": (stock,ticker),
        "hits": res['num_hits'],
        'status': res['meta']['status_code']
    })

stock_hit_dat = pd.json_normalize(hits_list)
stock_hit_dat.to_csv(get_project_root().joinpath("data/stock_hits.csv"))

In [18]:
stock_hit_dat

,stock,hits,status
0,"(Apple, AAPL)",375,200
1,"(Microsoft, MSFT)",138,200
2,"(Amazon, AMZN)",470,200
3,"(Tesla, TSLA)",244,200
4,"(Google, GOOGL)",422,200
5,"(Facebook, META)",2677,200
6,"(NVIDIA, NVDA)",22,200
7,"(JPMorgan, JPM)",51,200
8,"(Johnson & Johnson, JNJ)",94,200
9,"(Walmart, WMT)",69,200
